### Twogram Threegram Shared Word

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# sentence check
sentence_check = True  # True, False
#twogram_sentence_check = False  # True, False
#threegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/1-Twogram Threegram Shared Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))

In [ ]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{data_kind.capitalize()}_Result{file_ext}.xlsx", index=False)    

#### Concat Result

In [ ]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

In [ ]:
# Option For Frequency
if shared_word_frequency:
    #df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
    df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
    df_word.drop_duplicates(inplace=True)
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    pass

df_word

In [ ]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Result{file_ext}.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

In [ ]:
# Option For Twogram Sentence
#if twogram_sentence_check:
if sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
    df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
    df_shared_twogram.drop_duplicates(inplace=True)
    df_shared_twogram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_twogram

In [ ]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_Result{file_ext}.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

In [ ]:
# Option For Threegram Sentence
#if threegram_sentence_check:
if sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram_sent = df_threegram_sent.loc[:,["threegram"]]
    df_shared_threegram = pd.merge(df_shared_threegram, df_threegram_sent, how="inner", on="threegram")
    df_shared_threegram.drop_duplicates(inplace=True)
    df_shared_threegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_threegram

In [ ]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram], axis=1)
df_shared_all

In [ ]:
# Optional Output
#if sentence_check:
#    df_shared_all.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Threegram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_threegram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#else:
#    df_shared_all.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Threegram_All_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_All_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_threegram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_All_Result_With_Frequency{file_ext}.xlsx", index=False)

##### Concat Result With Comma

In [ ]:
# \******
df_word_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
df_word_prefix_suffix = df_word_prefix_suffix[["search_word","word"]]
df_word_prefix_suffix

In [ ]:
if file_ext == "5":
    df_all_word = df_word_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "6":
    df_word_not_prefix_suffix = df_word_all
    df_word_not_prefix_suffix["search_word"] = df_word_all["word"]
    df_word_not_prefix_suffix = df_word_not_prefix_suffix[["search_word","word"]]

    df_all_word = df_word_not_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
else:
    pass
    
df_all_word
# /******

In [ ]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [ ]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram")  

In [ ]:
# \*****
df_word_order_twogram = pd.merge(df_word_order_twogram,df_all_word, how="inner", on="word")
df_word_order_threegram = pd.merge(df_word_order_threegram,df_all_word, how="inner", on="word")

In [ ]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["search_word"])["twogram"].apply(", ".join).reset_index()  
df_word_order_join_threegram = df_word_order_threegram.groupby(["search_word"])["threegram"].apply(", ".join).reset_index()

In [ ]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram]

In [ ]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['search_word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all.rename(columns={"search_word":"word"}, inplace=True)
df_word_order_join_all 

In [ ]:
df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
df_word_order_join_all.drop_duplicates(inplace=True)
df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram"]]
df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
df_word_order_join_all.reset_index(drop=True, inplace=True)
df_word_order_join_all

In [ ]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [ ]:
df_word_all = df_word_all.loc[:,["word","frequency"]]
df_word_all["word"] = df_word_all["word"].apply(lambda x: lower_strip_func(x))
df_word_all

In [ ]:
df_twogram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_all = df_twogram_all.loc[:,["twogram","frequency"]]
df_twogram_all["twogram"] = df_twogram_all["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram_all

In [ ]:
df_threegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram_all = df_threegram_all.loc[:,["threegram","frequency"]]
df_threegram_all["threegram"] = df_threegram_all["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram_all

In [ ]:
df_shared_process_all = df_word_order_join_all
for i in range(len(df_shared_process_all)):
    # column result
    try:
        # column result
        df_two_var = pd.DataFrame(df_shared_process_all.loc[i,"twogram"].split(", "), columns=["twogram"])
        # merge with all
        df_two_var_merge = pd.merge(df_two_var, df_twogram_all, how="left", on="twogram")
        df_two_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_two_var_merge_select = df_two_var_merge.head(word_sample_num)
        # to list value
        df_two_var_list = df_two_var_merge_select["twogram"].to_list()
        # list join
        df_two_var_list_join = ", ".join(df_two_var_list)
        # change value
        df_shared_process_all.loc[i,"twogram"] = df_two_var_list_join
    except:
        pass
    try:
        # column result
        df_three_var = pd.DataFrame(df_shared_process_all.loc[i,"threegram"].split(", "), columns=["threegram"])
        # merge with all
        df_three_var_merge = pd.merge(df_three_var, df_threegram_all, how="left", on="threegram")
        df_three_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_three_var_merge_select = df_three_var_merge.head(word_sample_num)
        # to list value
        df_three_var_list = df_three_var_merge_select["threegram"].to_list()
        # list join
        df_three_var_list_join = ", ".join(df_three_var_list)
        # change value
        df_shared_process_all.loc[i,"threegram"] = df_three_var_list_join
    except:
        pass

df_shared_process_all
# /****

In [ ]:
df_word_order_join_twogram = df_shared_process_all[["word","frequency","twogram"]]
df_word_order_join_threegram = df_shared_process_all[["word","frequency","threegram"]]

In [ ]:
if sentence_check:
    df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Threegram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_threegram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
else:
    df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Threegram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_threegram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word*{file_ext}.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    #destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

### Twogram With Different Word List Pair

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 200  # native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 5

# word group start end
word_group1_start = word_start  # 0 native word start index
word_group1_end = 28 # 28  

word_group2_start = word_group1_end
word_group2_end = word_end # 200 native word end index


In [ ]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list, set_condition=False): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    set_condition used for drop duplicates in specific column 
    word_count_result(df, ["word","twogram"], set_condition=False):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/1-Twogram With Different Word List Pair/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram

In [ ]:
df_word_group1 = df_word_select.iloc[word_group1_start:word_group1_end,]
df_word_group1

In [ ]:
df_word_group2 = df_word_select.iloc[word_group2_start:word_group2_end,]
df_word_group2

In [ ]:
ngram_list = []
for j in df_word_group1["word"]:
    for k in df_word_group2["word"]:
        twogram_1_2 = f"{j} {k}"
        ngram_list.append(twogram_1_2)
        twogram_2_1 = f"{k} {j}"
        ngram_list.append(twogram_2_1)
df_pair_ngram = pd.DataFrame(ngram_list)
df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
df_pair_ngram.drop_duplicates(inplace=True)
df_pair_ngram

In [ ]:
df_word_groups_twogram = pd.merge(df_twogram, df_pair_ngram, how="inner", on="twogram")
df_word_groups_twogram.drop_duplicates(inplace=True)
df_word_groups_twogram.sort_values(by="frequency", ascending=False, inplace=True)
df_word_groups_twogram

In [ ]:
#df_word_groups_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_With_{word_group1_end}_And_{word_end-word_group1_end}_Word_Pair_All.xlsx", index=False)

In [ ]:
word_list = df_word_select["word"]

In [ ]:
# word usage in twogram condition
word_num_dict = {}
for i in word_list:
    word_num_dict[f"{i}"] = 0

result_list_select = []
var_list = []
for i in range(len(df_word_groups_twogram)):
    twogram = df_word_groups_twogram.loc[i,"twogram"]
    frequency = df_word_groups_twogram.loc[i,"frequency"]
    words = word_tokenize(twogram)   
    temp_list = [word for word in words]
    temp_list = temp_list + var_list
    # word count for max
    dict_list_count = Counter(temp_list)
    count_list = list(dict_list_count.values())
    # word count for min
    count_list2 = list(word_num_dict.values())

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        pass
    else:
        var_list = temp_list
        result_list_select.append([twogram,frequency]) 

        for item2 in dict_list_count.items(): 
            word_num_dict[item2[0]] = item2[1]        

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}")

In [ ]:
df_twogram_select = pd.DataFrame(result_list_select, columns=["twogram","frequency"])
df_twogram_select.sort_values(by="frequency", ascending=False, inplace=True)
df_twogram_select.reset_index(drop=True, inplace=True)
df_twogram_select

In [ ]:
# *** Option for remove specific 
df_remove = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/\
Deploy2 Result Manuel/{lang_folder.capitalize()}/Basics Cover the Most Master File 28-172 Word Twogram.xlsx")
df_remove 

In [ ]:
df_twogram_select = df_twogram_select[~df_twogram_select["twogram"].isin(df_remove["twogram"].to_list())]
df_twogram_select.reset_index(drop=True, inplace=True)
df_twogram_select
# *** Option

In [ ]:
word_count_result(df_twogram_select, df_twogram_select.columns[0:1], set_condition=False)

In [ ]:
df_twogram_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_With_{word_group1_end}_And_{word_end-word_group1_end}_Word_Pair_Selected.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_With_{word_group1_end}_And_{word_end-word_group1_end}_Word_Pair*.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    #destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

### Twogram Threegram With Selected Word List 

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 200  # native word end index

## word all usage in twogram
#word_use_num_min = 1  # word usage in selected twograms 
#word_use_num_max = 5

# word group start end
word_group1_start = word_start  # 0 native word start index
word_group1_end = 28 # 28  

word_group2_start = word_group1_end
word_group2_end = word_end # 200 native word end index

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/1-Twogram Threegram With Selected Word List/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [6]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [7]:
df_word_group1 = df_word_select.iloc[word_group1_start:word_group1_end,]
df_word_group1

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,o,5013838
7,ben,4908913
8,de,4880315
9,çok,4852169


In [8]:
df_word_group2 = df_word_select.iloc[word_group2_start:word_group2_end,]
df_word_group2

,word,frequency
28,iyi,2383224
29,tamam,2330314
30,her,2264013
31,benim,2229616
32,sana,2063064
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [10]:
df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [21]:
word_condition = 3  # 1,2,3; 1 for 28 word, 2 for 172 word, 3 for 28+172 = 200 word

word_list1 = df_word_group1["word"].to_list()  # 28 word
word_list2 = df_word_group2["word"].to_list()  # 200-28 172 word
word_list3 = df_word_select["word"].to_list()  # 200 word

if word_condition == 1:
    word_list_select = word_list1
    file_name_add = "28_Word"
elif word_condition == 2:
    word_list_select = word_list2
    file_name_add = "172_Word"
else:
    word_list_select = word_list3
    file_name_add = "200_Word"

print(f"{word_condition}\n{file_name_add}")

3
200_Word


In [22]:
# Twogram Func
d_list2  = df_twogram.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list_select:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
25473,efendim aynı,6
25474,ama ister,6
25475,üzgünüm kız,6
25476,seni selam,6


In [23]:
df_twogram_select.to_excel(f"{lang_folder.capitalize()}_Twogram_With_{file_name_add}.xlsx", index=False)

In [24]:
# Threegram Func
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list_select:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,başka bir şey,75193
4,hayır hayır hayır,69443
...,...,...
109758,gece de bana,10
109759,pekala orada dur,10
109760,olan ben bu,10
109761,gece daha onları,10


In [25]:
df_threegram_select.to_excel(f"{lang_folder.capitalize()}_Threegram_With_{file_name_add}.xlsx", index=False)

#### Copy Move And Delete

In [26]:
output_file3 = glob.glob(f"{lang_folder.capitalize()}_*_With_*.xlsx")
output_file3

['Turkish_Twogram_With_28_Word.xlsx',
 'Turkish_Threegram_With_28_Word.xlsx',
 'Turkish_Twogram_With_172_Word.xlsx',
 'Turkish_Threegram_With_172_Word.xlsx',
 'Turkish_Twogram_With_200_Word.xlsx',
 'Turkish_Threegram_With_200_Word.xlsx']

In [27]:
for m in output_file3:
    source = m # source directory
    #destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    destination = path
    shutil.copy2(source, destination)

In [28]:
for k in output_file3:
    try:
        os.remove(k)
    except:
        pass

### Threegram With Word And Twogram Pair

In [99]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [100]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [101]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 200  # native word end index

## word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 5

# word group start end
word_group1_start = word_start  # 0 native word start index
word_group1_end = 28 # 28  

word_group2_start = word_group1_end
word_group2_end = word_end # 200 native word end index

In [102]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list, set_condition=False): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    set_condition used for drop duplicates in specific column 
    word_count_result(df, ["word","twogram"], set_condition=False):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [103]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/1-Threegram With Word And Twogram Pair/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [104]:
df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [105]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [106]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [107]:
df_word_group1 = df_word_select.iloc[word_group1_start:word_group1_end,]
df_word_group1

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,o,5013838
7,ben,4908913
8,de,4880315
9,çok,4852169


In [108]:
df_word_group2 = df_word_select.iloc[word_group2_start:word_group2_end,]
df_word_group2

,word,frequency
28,iyi,2383224
29,tamam,2330314
30,her,2264013
31,benim,2229616
32,sana,2063064
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [116]:
word_condition = 2  # 1,2; 1 for 28 word and 200-28 = 172 word twogram , 2 for 172 word and 28 word twogram, 3 for 28+172 = 200 word

word_list1 = df_word_group1["word"].to_list()  # 28 word
word_list2 = df_word_group2["word"].to_list()  # 200-28 172 word


if word_condition == 1:
    word_list_select = word_list1
    file_name_add = "28_Word_And_172_Word_Twogram"
    df_twogram = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/1-Twogram Threegram With Selected Word List/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Twogram_With_172_Word.xlsx")
    twogram_list = df_twogram.iloc[:,0].to_list()
elif word_condition == 2:
    word_list_select = word_list2
    file_name_add = "172_Word_And_28_Word_Twogram"
    df_twogram = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/1-Twogram Threegram With Selected Word List/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Twogram_With_28_Word.xlsx")
    twogram_list = df_twogram.iloc[:,0].to_list()
else:
    file_name_add = "No group selected !!!"

print(f"{word_condition}\n{file_name_add}")

2
172_Word_And_28_Word_Twogram


In [117]:
ngram_list = []
for j in word_list_select:
    for k in twogram_list:
        twogram_1_2 = f"{j} {k}"
        ngram_list.append(twogram_1_2)
        twogram_2_1 = f"{k} {j}"
        ngram_list.append(twogram_2_1)
df_pair_ngram = pd.DataFrame(ngram_list)
df_pair_ngram.rename(columns={0:"threegram"}, inplace=True)
df_pair_ngram.drop_duplicates(inplace=True)
df_pair_ngram

,threegram
0,iyi bir şey
1,bir şey iyi
2,iyi değil mi
3,değil mi iyi
4,iyi ben de
...,...
245955,evet kadar ister
245956,ister gibi kadar
245957,gibi kadar ister
245958,ister çok hayır


In [118]:
df_threegram_pair = pd.merge(df_pair_ngram, df_threegram, how="inner", on="threegram")
df_threegram_pair.drop_duplicates(inplace=True)
df_threegram_pair.sort_values(by="frequency", ascending=False, inplace=True)
df_threegram_pair.reset_index(drop=True, inplace=True)
df_threegram_pair

,threegram,frequency
0,başka bir şey,75193
1,öyle değil mi,68442
2,sana bir şey,56713
3,böyle bir şey,51970
4,ben de öyle,41375
...,...,...
19858,zaman mı bir,10
19859,bir şey seninle,10
19860,zaman beni ve,10
19861,ama bir dostum,10


In [119]:
# Option
word_list = df_word_select["word"].to_list()

In [122]:
# word usage in threegram condition
word_num_dict = {}
for i in word_list:
    word_num_dict[f"{i}"] = 0

result_list_select = []
var_list = []
for i in range(len(df_threegram_pair)):
    threegram = df_threegram_pair.loc[i,"threegram"]
    frequency = df_threegram_pair.loc[i,"frequency"]
    words = word_tokenize(threegram)   
    temp_list = [word for word in words]
    temp_list = temp_list + var_list
    # word count for max
    dict_list_count = Counter(temp_list)
    count_list = list(dict_list_count.values())
    # word count for min
    count_list2 = list(word_num_dict.values())

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        pass
    else:
        var_list = temp_list
        result_list_select.append([threegram,frequency]) 

        for item2 in dict_list_count.items(): 
            word_num_dict[item2[0]] = item2[1]        

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}")

Max condition: False 
Min condition: True


In [123]:
df_threegram_pair = pd.DataFrame(result_list_select, columns=["threegram","frequency"])
df_threegram_pair.sort_values(by="frequency", ascending=False, inplace=True)
df_threegram_pair.reset_index(drop=True, inplace=True)
df_threegram_pair

,threegram,frequency
0,başka bir şey,75193
1,öyle değil mi,68442
2,sana bir şey,56713
3,böyle bir şey,51970
4,ben de öyle,41375
...,...,...
65,senin gibi beni,49
66,herkes gibi beni,39
67,gibi onu geri,35
68,biri beni seni,30


In [115]:
word_count_result(df_threegram_pair, df_threegram_pair.columns[0:1], set_condition=False)
# Option

,word,word_count
0,ama,5
1,değil,5
2,onu,5
3,tabii,5
4,sen,5
...,...,...
126,biliyorum,1
127,bizim,1
128,buna,1
129,bunun,1


In [68]:
df_threegram_pair.to_excel(f"{lang_folder.capitalize()}_Threegram_With_{file_name_add}.xlsx", index=False)

#### Copy Move And Delete

In [69]:
output_file4 = glob.glob(f"{lang_folder.capitalize()}_Threegram_With_*.xlsx")
output_file4

['Turkish_Threegram_With_28_Word_And_172_Word_Twogram.xlsx',
 'Turkish_Threegram_With_172_Word_And_28_Word_Twogram.xlsx']

In [72]:
for n in output_file4:
    source = n # source directory
    #destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    destination = path
    shutil.copy2(source, destination)

In [73]:
for l in output_file4:
    try:
        os.remove(l)
    except:
        pass

In [120]:
def word_usage_check(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_threegram_pair.sort_values(by="frequency", ascending=False, inplace=True)
    df_threegram_pair.reset_index(drop=True, inplace=True)

    
    return df_result

In [121]:
word_usage_check(word_list, df_threegram_pair, "threegram", "frequency", 1, 5)

,threegram,frequency
0,başka bir şey,75193
1,öyle değil mi,68442
2,sana bir şey,56713
3,böyle bir şey,51970
4,ben de öyle,41375
...,...,...
65,senin gibi beni,49
66,herkes gibi beni,39
67,gibi onu geri,35
68,biri beni seni,30


In [124]:
def word_group_with_pair(word_group_pair1, word_group_pair2):
    ngram_list = []
    for j in word_group_pair1:
        for k in word_group_pair2:
            twogram_1_2 = f"{j} {k}"
            ngram_list.append(twogram_1_2)
            twogram_2_1 = f"{k} {j}"
            ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"threegram"}, inplace=True)
    df_pair_ngram.drop_duplicates(inplace=True)
    
    return df_pair_ngram

In [ ]:
word_group_with_pair(word_list1, word_list2)